<a href="https://colab.research.google.com/github/jubileu001/empregos_ce_basedosdados/blob/main/empregos_ce_basedosdados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Acessando o datalake da Base dos Dados

In [2]:
import basedosdados as bd
import pandas as pd

mudando o método de visualização base do pandas, tirando o matplotlib e colocando o plotly

In [3]:
pd.options.plotting.backend = 'plotly'

Se é a primeira vez fazendo acesso ao datalake, é necessario instalar a basedados. Basta utilizar !pip install basedosdados

Aqui vamos começar a carregar a tabela diretamente do datalake da Base dos Dados. Utilizaremos o modo com é mostrado no tutorial disponibilizado no github do projeto.

O modo de acesso apresentado é utilizando uma query SQL para realizar a consulta diretamente do python. Dessa forma, é definida a query e depois utilizado a modulo read_sql para acessar o datalake e realizar a consulta. billing_project_id é o id do projeto criado dentro do BigQuery.

A query criada tem como objetivo consulta a quantidade de vinculos criados e quantidade da população para cada ano.

In [32]:
query =  '''
  with vinculos_ceara as (
  select id_municipio, ano, count(*) numero_de_vinculos
  from basedosdados.br_me_rais.microdados_vinculos
  where sigla_uf = 'CE'
  group by ano, id_municipio
)

select t1.ano, SUM(t1.numero_de_vinculos) as numero_de_vinculos
from vinculos_ceara t1
join basedosdados.br_ibge_populacao.municipio t2
on t1.id_municipio = t2.id_municipio
and t1.ano = t2.ano
group by t1.ano'''

df = bd.read_sql(query, billing_project_id='<billing-project-id>')

Downloading: 100%|██████████| 30/30 [00:00<00:00, 105.75rows/s]


Assim temos carregado a tabela na variável df e posterior vamos gerar um gráfico que apresente o número de vinculos gerados em cada ano, de 1991 a 2020

In [33]:
df.head(10)

,ano,numero_de_vinculos
0,1991,741120
1,1992,717479
2,1993,717724
3,1994,757247
4,1995,831917
5,1996,833450
6,1997,854141
7,1998,872644
8,1999,875415
9,2000,923984


In [34]:
df.plot(x='ano', y='numero_de_vinculos')

# Outro método de acesso

Do meu ponto de vista é mais interessante realizar a consulta SQL diretamente na plataforma do google cloud e salvar a tabela para assim ter acesso já a tabela pronta. Acredito que seja a melhor opção, uma vez que eu particularmente não gosto muito de gerar query sql dentro do python. Mas o faria se realmente necessario.

Dessa forma, a mesma query criada acima foi utilizada dentro da plataforma do google cloud e gerada a tabela 'empregos_ce_table'. Mundando o atributo 'query_project_id' para o projeto onde contem a tabela criada, esse atributo vem prenchido como 'basedosdados', podemos fazer o acesso a tabela criada.

In [27]:
df2 = bd.read_table('empregos_ce','empregos_ce_table',query_project_id='bd-teste-1',billing_project_id='billing_project_id')


Downloading: 100%|██████████| 30/30 [00:00<00:00, 165.66rows/s]


In [30]:
df2.head(10)

,ano,numero_de_vinculos,populacao
0,1991,741120,5980012
1,1992,717479,6255016
2,1993,717724,6403244
3,1994,757247,6536241
4,1995,831917,6708285
5,1996,833450,6809290
6,1997,854141,6920292
7,1998,872644,7013376
8,1999,875415,7106605
9,2000,923984,7431597


In [31]:
df2.plot(x='ano', y='numero_de_vinculos')

## Conclusão

Foram apresentados dois modos de acesso ao datalake da base de dados e feita a consulta a dois bancos de dados disponibilidos por eles.

Por fim, pode-se observar que a partir de 1995 é possivel notar um aumento no número de vinculos no estado do Ceará. Esse aumento é mais perceptivel a partir do ano 2000 e começa a cair em 2014, com maior queu de 2015 para 2016. Em os anos de 2018 e 2019 é visto um pequeno aumento mas esse valor cai novamente no ano de 2020.